In [21]:
import sys # para movernos en carpeta dentro del S.O
import os  # movernos por carpetas|
import numpy as np #generación y manejo de datos
from keras.utils import plot_model #genérica de gráfica 
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Para tratamiento de imagenes
from tensorflow.keras import optimizers #obtener optimizador
from tensorflow.keras.models import Sequential #redes neuronales secuenciales, capas en orden
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D 
from tensorflow.keras.callbacks import  TensorBoard ##Plataforma para visualizar las graficas
import tensorflow as tf #cálculo numérico

In [22]:
###Cargamos nuestros Data
data_entrenamiento = './Plagas_Arroz/train'
data_validacion = './Plagas_Arroz/test'

###Parametros de la red neuronal
epocas = 10 #cantidad de iteracion
altura,longitud = 150 , 150 # tamaño de las imagenes pixeles
batch_size = 32 #numero de imagenes a procesar en cada paso
pasos = 1000   #numero de veces q se procesa la informacion en las epocas
filtrosConv1 = 32 #numero de filtro
filtrosConv2 = 64 #numero de filtro
filtrosConv3 = 128 #numero de filtro
filtrosConv4 = 256 #numero de filtro
tamano_filtro1 = (3, 3) #Kernel 
tamano_filtro2 = (2, 2) #Kernel 
tamano_filtro3 = (2, 2) #Kernel 
tamano_filtro4 = (2, 2) #Kernel 
tamano_pool = (2, 2) #tamaño del filtro del Maxpoling
clases = 9  #numero de clases
lr = 0.001 #learning rate ajuste de la red neuronal

###Pre_Procesamiento de Imagenes

In [23]:
##Prepocesamiento de iamgenes para luego usarlas en la red 

#Entrenamiento
#Para que haya mejor entrenamiento en la imagenes
entrenamiento_datagen = ImageDataGenerator( 
    rescale= 1. / 255,     #todos lo pixels poseen rango de 0-255 #Reescalado cada pixel de 0 a 1 mejor eficiencia
    shear_range=0.2,       #genera imagenes en inclinacion
    zoom_range=0.2,        #zoom a las imagenes
    horizontal_flip=True)  #invertir imagenes 

##Validacion
#para la validacion solo reescalamos con el fin iran las imagenes como son 
validacion_datagen = ImageDataGenerator(
    rescale= 1. / 255)      #cada pixel de 0 a 1 mejor eficiencia

###Entrenamiento

In [24]:
#entra al directorio y procesa imagenes segun los datos dados
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,## se extrae imagenes 
    target_size=(altura, longitud), #todas la iamgenes a una altura y longitud dada
    batch_size=batch_size,  #se procesa con un batch size de 32 
    class_mode='categorical',)##se usa porque la clasificacion es por categoria

##Validacion
imagen_validacion= validacion_datagen .flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 2700 images belonging to 9 classes.
Found 450 images belonging to 9 classes.


In [25]:
###Ejemplo de una plaga normalizada
from PIL import Image
im = Image.open('normalizada.jpg')
im.show()

###CReacion de Red convolucion 

##Estructura de la red neuronal

In [26]:
#Estructura de la REd

model = Sequential() ##Red secuencial, es decir varias capas apiladas

##Primera capa con 32 filtro, 3 canales rgb
#SAME  la salida tenga el mismo tamaño que la entrada
#Relu anula los valores negativos a positivo
##«2×2» (2 de alto por 2 de ancho = 4 pixeles)

##Capa de entrada
model.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=tamano_pool))


##Capa Oculta
##segunda capa con 64 filtro
model.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same", activation='relu'))
model.add(MaxPooling2D(pool_size=tamano_pool))


##Tercera capa con 128 filtro
model.add(Convolution2D(filtrosConv3, tamano_filtro3, padding ="same", activation='relu'))
model.add(MaxPooling2D(pool_size=tamano_pool))


model.add(Flatten())  ##iamgen se aplana y devuelve una matriz unidimensional
##256 neuronas es la topologia de la red
model.add(Dense(256, activation='relu'))  #capa dense para conectar 256 neuronas en una sola capa
model.add(Dropout(0.5))     ##apagar neuronas a la capa dense el 50% evitar sobre ajuste


############Capa de salida
model.add(Dense(clases, activation='softmax')) #9 neuronas, sotfmax obtener probabilidades 

#'categorical_crossentropy' para ver q tambien o q mal ira 
model.compile(loss='categorical_crossentropy', ## parametro para optimizar el algoritmo
            optimizer=optimizers.Adam(lr), ##optimizador Adam
            metrics=['accuracy'])    ## porcentaje de como aprende nuestra red neuronal 

##entrenar algoritmo

In [27]:
#entrenar red neruonal con imagenes de entrenamiento
model.fit(imagen_entrenamiento,
    epochs=epocas,
    validation_data = imagen_validacion,
    steps_per_epoch = imagen_entrenamiento.n//batch_size, #se optiene los pasos mas optimo, 84 pasos
    validation_steps = imagen_validacion.n //batch_size,   #se procesa 32 imagenes por paso, 84 pasos
    callbacks = [TensorBoard (log_dir = 'logs/Grafica#18')],
    )

Epoch 1/10
49/84 [================>.............] - ETA: 15s - loss: 2.2462 - accuracy: 0.1728

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


84/84 [==============================] - 47s 553ms/step - loss: 2.1456 - accuracy: 0.2083 - val_loss: 1.7278 - val_accuracy: 0.3772
Epoch 2/10
84/84 [==============================] - 47s 561ms/step - loss: 1.6710 - accuracy: 0.3966 - val_loss: 1.5075 - val_accuracy: 0.4866
Epoch 3/10
84/84 [==============================] - 39s 468ms/step - loss: 1.3300 - accuracy: 0.5337 - val_loss: 1.0873 - val_accuracy: 0.6496
Epoch 4/10
84/84 [==============================] - 38s 447ms/step - loss: 1.0437 - accuracy: 0.6409 - val_loss: 0.8105 - val_accuracy: 0.7589
Epoch 5/10
84/84 [==============================] - 40s 472ms/step - loss: 0.8768 - accuracy: 0.7121 - val_loss: 0.7528 - val_accuracy: 0.7946
Epoch 6/10
84/84 [==============================] - 40s 472ms/step - loss: 0.7079 - accuracy: 0.7654 - val_loss: 0.6203 - val_accuracy: 0.8304
Epoch 7/10
84/84 [==============================] - 40s 479ms/step - loss: 0.5785 - accuracy: 0.8047 - val_loss: 0.5050 - val_accuracy: 0.8638
Epoch 8/10

In [28]:
#obtner informacion de la red neuronal
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 75, 75, 64)        8256      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 37, 37, 128)       32896     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 18, 18, 128)     

In [29]:
#sirve para poder graficar la estructura de la red
tf.keras.utils.plot_model(model,
                          to_file='nombre-nn.png',
                          show_shapes=False,
                          show_dtype=False,
                          show_layer_names=True,
                          rankdir='TB',
                          expand_nested=False,
                          dpi=96,
                          layer_range=None,
                          show_layer_activations=True) 

" tf.keras.utils.plot_model(model,\n                          to_file='nombre-nn.png',\n                          show_shapes=False,\n                          show_dtype=False,\n                          show_layer_names=True,\n                          rankdir='TB',\n                          expand_nested=False,\n                          dpi=96,\n                          layer_range=None,\n                          show_layer_activations=True) "

###Guardar modelo en un archivo

In [30]:
###Se guarda el modelo 
target_dir = './Modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./Modelo/modelo.P52')
model.save_weights('./Modelo/pesos.P52')

INFO:tensorflow:Assets written to: ./Modelo/modelo.P52\assets


INFO:tensorflow:Assets written to: ./Modelo/modelo.P52\assets
